In [93]:
import numpy as np 
import pandas as pd
import sklearn


In [94]:
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Scikit-learn version:", sklearn.__version__)

NumPy version: 1.26.4
Pandas version: 2.2.2
Scikit-learn version: 1.4.2


In [ ]:
df = pd.read_csv('spam.csv',encoding='latin1')

## 1.Preprocessing 

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df[df['v1']=='spam']['v2']

In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [ ]:
df.sample(5)

In [ ]:
# rename
df.rename(columns={'v1':'target', 'v2':'text'},inplace=True)


In [ ]:
#lable incoding

from sklearn.preprocessing import LabelEncoder

encoder =LabelEncoder()

In [ ]:
df['target']=encoder.fit_transform(df['target'])

In [ ]:
df.sample(5)

In [ ]:
# missing 
df.isnull().sum()

In [ ]:
# cheak for duplicated value
df.duplicated().sum()

In [ ]:
# remove duplicats
df=df.drop_duplicates(keep='first')

In [ ]:
# cheak for duplicated value
df.duplicated().sum()

In [ ]:
## EDA 


In [ ]:
count=df['target'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.pie(count,labels=['mah','spam'],autopct='%1.1f%%')

In [ ]:
! pip install nltk

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
df['num_character'] = df['text'].apply(len)


In [ ]:
df.head()

In [ ]:
# number of words
df['num_word']=df['text'].apply(lambda x:len(nltk.word_tokenize(x)))

In [ ]:
df.head()

In [ ]:
# number of sentences
df['num_sentence']=df['text'].apply(lambda x:len(nltk.sent_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df[['num_character', 'num_word', 'num_sentence']].describe()


In [ ]:
df[df['target']==0][['num_word','num_character','num_sentence']].describe()

In [ ]:
df[df['target']==1][['num_word','num_character','num_sentence']].describe()

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(12,10))
sns.histplot(df[df['target']==0]['num_character'])
sns.histplot(df[df['target']==1]['num_character'],color='red')

In [ ]:
plt.figure(figsize=(12,10))
sns.histplot(df[df['target']==0]['num_word'])
sns.histplot(df[df['target']==1]['num_word'],color='red')

In [ ]:
plt.figure(figsize=(12,10))
sns.histplot(df[df['target']==0]['num_sentence'])
sns.histplot(df[df['target']==1]['num_sentence'],color='red')

In [ ]:
sns.pairplot(df,hue='target')

In [ ]:
numeric_df = df.select_dtypes(include='number')

# Create the heatmap
sns.heatmap(numeric_df.corr(), annot=True)

In [ ]:
##3. Data Preprocessing
# Lower case
#Tokenization
#Removing special characters
#Removing stop words and punctuation
#Stemming

In [ ]:
import nltk
nltk.download('stopwords')


In [ ]:
import string
from nltk.corpus import stopwords
import nltk

# Download stopwords if not already downloaded
nltk.download('stopwords')

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
#from nltk.corpus import stopwords

#stopwords.words('english')


#import string
#string.punctuation

In [ ]:
def tarnsform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    y=[]
    for  i in text:
        if i.isalnum():
            y.append(i)
    text = y[:]
    y.clear()
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    text = y[:]
    y.clear()

    for i in text:
        y.append(ps.stem(i))
    return " ".join(y)       

In [ ]:
tarnsform_text('hii are aindfkbad  sjddb 34 $$nnsdkdjfns bnjsd loving')

In [ ]:
df['transformed_text']=df['text'].apply(tarnsform_text)

In [ ]:
df.head()

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(width=500,height=500,min_font_size=10,background_color='white')

In [ ]:
spam_wc=wc.generate(df[df['target']==1]['transformed_text'].str.cat(sep=" "))

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(spam_wc)

In [ ]:
ham_wc=wc.generate(df[df['target']==0]['transformed_text'].str.cat(sep=" "))

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(ham_wc)

In [ ]:
df.head()

In [ ]:
#df[df['target']==1]['transformed_text'].tolist()

In [ ]:
spam_corpur = []
for msg in df[df['target']==1]['transformed_text'].tolist():
    for word in msg.split():
        spam_corpur.append(word)



In [ ]:
len(spam_corpur)

In [ ]:
from collections import Counter
sns.barplot(x=pd.DataFrame(Counter(spam_corpur).most_common(30))[0],y=pd.DataFrame(Counter(spam_corpur).most_common(30))[1])
plt.xticks(rotation='vertical')

In [ ]:
ham_corpur = []
for msg in df[df['target']==0]['transformed_text'].tolist():
    for word in msg.split():
        ham_corpur.append(word)

In [ ]:
len(ham_corpur)

In [ ]:
from collections import Counter
sns.barplot(x=pd.DataFrame(Counter(ham_corpur).most_common(30))[0],y=pd.DataFrame(Counter(ham_corpur).most_common(30))[1])
plt.xticks(rotation='vertical')

In [ ]:
df.head()

 ## 4.Model Building

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer()
tfidf=TfidfVectorizer(max_features=3000)

In [ ]:
x=tfidf.fit_transform(df['transformed_text']).toarray()

In [ ]:
df.shape

In [ ]:
y=df['target'].values

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

gnb=GaussianNB()
mnb=MultinomialNB()
bnb=BernoulliNB()

In [ ]:
gnb.fit(x_train,y_train)
y_pred1=gnb.predict(x_test)
print(accuracy_score(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))
print(precision_score(y_test,y_pred1))


In [ ]:
mnb.fit(x_train,y_train)
y_pred2=mnb.predict(x_test)
print(accuracy_score(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))
print(precision_score(y_test,y_pred2))

In [ ]:
bnb.fit(x_train,y_train)
y_pred3=bnb.predict(x_test)
print(accuracy_score(y_test,y_pred3))
print(confusion_matrix(y_test,y_pred3))
print(precision_score(y_test,y_pred3))

In [ ]:
# tfidf == mnb

In [ ]:
pip install xgboost


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier


In [ ]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)

In [ ]:
clfs = {
    'SVC' : svc,
    'KN' : knc, 
    'NB': mnb, 
    'DT': dtc, 
    'LR': lrc, 
    'RF': rfc, 
    'AdaBoost': abc, 
    'BgC': bc, 
    'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
}

In [ ]:
def train_classifier(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    
    return accuracy,precision

In [ ]:
train_classifier(svc,x_train,y_train,x_test,y_test)

In [ ]:
accuracy_scores = []
precision_scores = []

for name,clf in clfs.items():
    
    current_accuracy,current_precision = train_classifier(clf, x_train,y_train,x_test,y_test)
    
    print("For ",name)
    print("Accuracy - ",current_accuracy)
    print("Precision - ",current_precision)
    
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)

In [ ]:
performance_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy':accuracy_scores,'Precision':precision_scores}).sort_values('Precision',ascending=False)

In [ ]:
performance_df1 = pd.melt(performance_df, id_vars = "Algorithm")

In [ ]:
performance_df1

In [ ]:
sns.catplot(x = 'Algorithm', y='value', 
               hue = 'variable',data=performance_df1, kind='bar',height=5)
plt.ylim(0.5,1.0)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
import pickle 
pickle.dump(tfidf,open('vectorizer.pkl','wb'))
pickle.dump(mnb,open('model.pkl','wb'))

In [ ]:
import pickle

# Save the trained objects
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

with open('model.pkl', 'wb') as f:
    pickle.dump(mnb, f)
